<h1 align="center">Scientific Programming in Python</h1>
<h2 align="center">Topic 5: Accelerating Python with Cython: Writting C in Python </h2> 


_Notebook created by Martín Villanueva - `martin.villanueva@usm.cl` - DI UTFSM - May2017._

In [1]:
import numba
import numpy as np
from math import sqrt

%load_ext Cython

## La distancia de Hausdorff nuevamente...

En esta actividad volveremos a implementar la distancia/métrica de Hausdorff, pero ahora utilizando Cython.

__La métrica de Hausdorff__ corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 

Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.
<img src='data/hausdorff.png' style="width: 600px;">

A continuación se le proveen 3 funciones que implementan tal métrica, usando __Numba__.

In [2]:
@numba.jit('float64 (float64[:], float64[:])')
def metric_numba(x, y):
    """
    standard Euclidean distance
    """
    ret = x-y
    ret *= ret
    return np.sqrt(ret).sum()


@numba.jit('float64 (float64[:], float64[:,:])', nopython=True)
def inf_dist_numba(x, Y):
    """
    inf distance between row x and array Y
    """
    m = Y.shape[0]
    inf = np.inf
    
    for i in range(m):
        dist = metric_numba(x, Y[i])
        if dist < inf:
            inf = dist
    return inf

@numba.jit('float64 (float64[:,:], float64[:,:])', nopython=True)
def hausdorff_numba(X, Y):
    """
    Hausdorff distance between arrays X and Y
    """
    m = X.shape[0]
    n = Y.shape[0]
    sup1 = -1.
    sup2 = -1.
    
    for i in range(m):
        inf1 = inf_dist_numba(X[i], Y)
        if inf1 > sup1:
            sup1 = inf1
    for i in range(n):
        inf2 = inf_dist_numba(Y[i], X)
        if inf2 > sup2:
            sup2 = inf2
            
    return max(sup1, sup2)

Se solicita que realice lo siguiente:

1. Escribir el equivalente __Cython__ de las tres funciones anteriores, ocupando todas las optimizaciones posibles: __Compiler directives__, __Memory Views__, __Inline Functions__, __Pure C functions__ o cualquier otra optimización que usted considere conveniente.
2. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las versiones __Numba__ y __Cython__ de las funciontes anteriores sobre estos arreglos.
3. Concluya.

In [3]:
%%cython -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp
from libc.math cimport sqrt

ctypedef cnp.float64_t float64_t

cdef float64_t metric_cython(float64_t[::1] x, float64_t[::1] y):
    """
    standard Euclidean distance
    """
    cdef:
        int m = x.shape[0]
        int i = 0
        float ret = 0
    for i in range(m):
        ret += (x[i]-y[i])**2
    return sqrt(ret)

cdef inline float64_t inf_dist_cython(float64_t[::1] x, float64_t[:,::1] Y):
    """
    inf distance between row x and array Y
    """
    cdef:
        int rows = Y.shape[0]
        float inf = np.inf
        float dist = 0.0
        int i = 0
    
    for i in range(rows):
        dist = metric_cython(x, Y[i])
        if dist < inf:
            inf = dist
    return inf

def hausdorff_cython(float64_t[:,::1] X, float64_t[:,::1] Y):
    """
    Hausdorff distance between arrays X and Y
    """
    cdef:
        int i = 0
        int m = X.shape[0]
        int n = Y.shape[0]
        float sup1 = -1
        float sup2 = -1
        float inf1
        float inf2
    
    for i in range(m):
        inf1 = inf_dist_cython(X[i], Y)
        if inf1 > sup1:
            sup1 = inf1
            
    for i in range(n):
        inf2 = inf_dist_cython(Y[i], X)
        if inf2 > sup2:
            sup2 = inf2
    if sup1>sup2:
        return sup1
    return sup2

In [4]:
Xs = []
Ys = []
for i in range(1,11):
    i *= i
    X = np.random.rand(i*100*3).reshape(i*100,3)*100
    Y = np.random.rand(i*120*3).reshape(i*120,3)*100
    Xs.append(X)
    Ys.append(Y)

In [5]:
pruebas = dict()
for i in range(10):
    print("Experimento",i+1)
    print("X:({0},{1}),Y:({2},{1})".format(100*(i+1)*(i+1),3,120*(i+1)*(i+1)))
    Xi = Xs[i]
    Yi = Ys[i]
    print("Numba:")
    cls = %timeit -o hausdorff_numba(Xi, Yi)
    print("Cython:")
    num = %timeit -o hausdorff_cython(Xi,Yi)
    pruebas[i] = (cls, num)

Experimento 1
X:(100,3),Y:(120,3)
Numba:
10 loops, best of 3: 22 ms per loop
Cython:
1000 loops, best of 3: 1.65 ms per loop
Experimento 2
X:(400,3),Y:(480,3)
Numba:
1 loop, best of 3: 340 ms per loop
Cython:
10 loops, best of 3: 25.4 ms per loop
Experimento 3
X:(900,3),Y:(1080,3)
Numba:
1 loop, best of 3: 1.72 s per loop
Cython:
10 loops, best of 3: 128 ms per loop
Experimento 4
X:(1600,3),Y:(1920,3)
Numba:
1 loop, best of 3: 5.44 s per loop
Cython:
1 loop, best of 3: 406 ms per loop
Experimento 5
X:(2500,3),Y:(3000,3)
Numba:
1 loop, best of 3: 13.3 s per loop
Cython:
1 loop, best of 3: 990 ms per loop
Experimento 6
X:(3600,3),Y:(4320,3)
Numba:
1 loop, best of 3: 29.2 s per loop
Cython:
1 loop, best of 3: 2.16 s per loop
Experimento 7
X:(4900,3),Y:(5880,3)
Numba:
1 loop, best of 3: 52 s per loop
Cython:
1 loop, best of 3: 3.86 s per loop
Experimento 8
X:(6400,3),Y:(7680,3)
Numba:
1 loop, best of 3: 1min 29s per loop
Cython:
1 loop, best of 3: 6.74 s per loop
Experimento 9
X:(8100,3),Y